In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt               

In [2]:
dataset = dgl.data.CSVDataset('../../Dati/Dataset_Binario/')
dataset

DGLError: 'meta.yaml' cannot be found under ../../Dati/Dataset_Binario/.

In [ ]:
numer_class = []
a = 0
b = 0 
c = 0
d = 0
e = 0
f = 0
gg = 0
h = 0
for g, i in dataset:
    if i["label"].item() not in numer_class:
        numer_class.append(i["label"].item())
    if i["label"].item() == 0:
        a += 1
    elif i["label"].item() == 1:
        b += 1
    elif i["label"].item() == 2:
        c += 1
    elif i["label"].item() == 3:
        d += 1
    elif i["label"].item() == 4:
        e += 1
    elif i["label"].item() == 5:
        f += 1
    elif i["label"].item() == 6:
        gg += 1
    elif i["label"].item() == 7:
        h += 1

print(a, b, c, d, e, f, gg, h, (a+b+c+d+e+f+gg+h))
print(len(numer_class))

In [ ]:
dim_nfeats = 2
gclasses = len(numer_class)

In [ ]:
size_batch = len(numer_class)*2
size_batch

In [ ]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size= size_batch, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size= size_batch, drop_last=False)

In [ ]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

In [ ]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())

# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [ ]:
if gclasses == 2:
    num_of_epoches = 200
elif gclasses >= 2:
    num_of_epoches = 1000

## Analisi risultati
Con il dataset Binario l'accuratezza si aggira attorno al 0.9.
Con il dataset Multiplo il quale momentaneamente contiene 4 classi di grafi l'accuratezza si aggira attorno al 0.54.

## PROVE


Numero di nodi 16

batch = 10 epoche 1000 con soglia 400epoche e loss < 0.4 -> ***ACCURATEZZA OTTENUTA*** 0,61

batch = 8 epoche 5000  -> ***ACCURATEZZA OTTENUTA*** 0,61 con ultima loss -> 0.7

batch = 8 epoche 1000  -> ***ACCURATEZZA OTTENUTA*** 0,59 con ultima loss -> 0.87

batch = 8 epoche 1000  -> ***ACCURATEZZA OTTENUTA*** 0,63 con ultima loss -> 1.61





Numero di nodi 32

batch = 8 epoche 1000 ->  ***ACCURATEZZA OTTENUTA*** 0,62 con ultima loss ->  1.09


Numero di nodi 12

batch = 8 epoche 1000 ->  ***ACCURATEZZA OTTENUTA*** 0,62 con ultima loss ->   0.75







Numero di nodi 12 classificazione di sei classi 

batch = 12 epoche 1000 -> ***ACCURATEZZA OTTENUTA*** 0,42 con ultima loss ->  0.84

In [ ]:
import torch
import numpy as np

from os.path import join
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import accuracy_score

class AverageValueMeter():
    """ Compute the average loss between batches.
    """
    
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.sum = 0
        self.num = 0
        
    def add(self, value, num):
        self.sum += value * num
        self.num += num
        
    def value(self):
        try: 
            return self.sum / self.num
        except:
            return None
        

def train_classifier(model: nn.Module, 
                     train_loader, 
                     test_loader, 
                     exp_name="experiment", 
                     lr=0.001, 
                     epochs=10, 
                     momentum=0.9, 
                     logdir="logs"):
    """ Train the model.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_meter = AverageValueMeter()
    accr_meter = AverageValueMeter()
    writer = SummaryWriter(join(logdir, exp_name))
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    loader = {
        'train': train_loader, 
        'test':  test_loader
    }
    
    global_step = 0
    for e in tqdm(range(epochs)): 
        for mode in ['train', 'test']:
            loss_meter.reset()
            accr_meter.reset()
            model.train() if mode == 'train' else model.eval()
            with torch.set_grad_enabled(mode=='train'):
                for i, batch in enumerate(loader[mode]):
                    
                    
                    #batched_graph, batched_graph.ndata['label'].float()
                    
                    x = batch[0]#.to(device)
                    y = batch[1]['label'].long()#.to(device)
                    #print(y['label'])
                    
                    output = model(x, x.ndata['label'].float())
                    
                    n = size_batch # elements in batch
                    global_step += n
                    l = criterion(output, y)
                    
                    if mode == 'train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                        
                    acc = accuracy_score(
                        y.to('cpu'), 
                        output.to('cpu').max(1)[1]
                    )
                    loss_meter.add(l.item(), n)
                    accr_meter.add(acc, n)
                    
                    if mode == 'train':
                        writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                        writer.add_scalar('accuracy/train', accr_meter.value(), global_step=global_step)
            
            writer.add_scalar('loss/test', loss_meter.value(), global_step=global_step)
            writer.add_scalar('accuracy/test', accr_meter.value(), global_step=global_step)
        torch.save(model.state_dict(), '%s-%d.pth' % (exp_name, e+1))
    return model
    
    
def test_classifier(model, loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    predictions, labels = [], []
    for batch in loader:
        x = batch[0]#.to(device)
        y = batch[1]['label'].long()#.to(device)
        output = model(x, x.ndata['label'].float())
        preds = output.to('cpu').max(1)[1].numpy()
        labs = y.to('cpu').numpy()
        predictions.extend(list(preds))
        labels.extend(list(labs))
    return np.array(predictions), np.array(labels)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
model2 = GCN(dim_nfeats, 12, gclasses)

model2 = train_classifier(
    model2,
    train_dataloader,
    test_dataloader,
    'gnn_experiment',
    epochs = num_of_epoches
)

In [ ]:
classificazione = test_classifier(model2, test_dataloader)
print(classificazione)

In [ ]:
accuracy_score(classificazione[0], classificazione[1])